In [33]:
from math import cos, sin, pi

WB = 2                            # RHP zero, wheel base

class Sphero(object):
    """ Creating a Sphero object for the sample maze construction. The steering angle is assumed to be small to make the 
assumption in calculations. The numerical calculations, as a result, are stable. In future, two other models like point mass 
and SO(3) will also be added if needed.
1. In the first model, we assumed a segway model which has the same inputs as the actual robot but the measurements are not 
available as position or orientation
2. In the second model, we assume a point mass model where the acceleration is the input to the robot as well as the measurements
in the form of change in acceleration.
3. The third model is a mix of both the above models, that represents an input-output model of the robot. The input to the 
system is the heading and velocity (segway) and the output is the collision in the form of change in acceleration. The segway 
model is simulated and the states are differentiatied to obtain V and A. With added noise in A, we integrate back to get the 
position of the robot. If collision is detected, we add this to the integrated data.

Created by Srinivas K."""
    def __init__(self):
        pass
    
    def motion(self, pose, control, dt):  # Unicycle mkotion model
        control[1]=control[1]%360
        angle = ((control[1]+pose[2])%360)*pi/180
        return ([x+y for x,y in zip(pose,[control[0]*dt*cos(angle), 
                                          control[0]*dt*sin(angle), 
                                          control[0]*dt*sin(control[1]*pi/180)/WB])])
#         pose[0]=pose[0]+control[0]*dt*cos(control[1]*pi/180+pose[2])
#         pose[1]=pose[1]+control[0]*dt*sin(control[1]*pi/180+pose[2])
#         pose[2]=pose[2]+control[0]*dt*sin(control[1]*pi/180)/WB
        
        return pose
    
    def draw(self, pose):     # Draw the Sphero robot in the simulated scene
        if self.cursor_objects:
            map(self.canvas.delete, self.cursor_objects)
            self.cursor_objects = []
        
        xsp, ysp =[], []
        xsp.append(pose[0])
        ysp.append(pose[1])
        for i in arange(0,360,1):      # List of spherical points
            xsp.append(pose[0] + r * cos(i*pi/180+(pose[2]*180/pi)))
            ysp.append(pose[1] + r * sin(i*pi/180+(pose[2]*180/pi)))
        
        f = (xsp,ysp)
        
        for i in range(len(f[0])):
            self.cursor_objects.append(self.canvas.create_line(f[0][(i)%len(f[0])],f[1][(i)%len(f[0])],
                                                         f[0][(i+1)%len(f[0])],f[1][(i+1)%len(f[0])]))
    
    # Constraint set in a GUI window
    def Sphero_constraint(self,origin,ww,ws,wl):
        a = origin[0]
        b = origin[1]
        
        # Creating boundary set (inclusing the wall width and the radius of the ball)
        bc = []  # In the form of a^Tx<=b where x=[X Y] and bc = [X Y b]
        bc.append([1, 0, a+ww+r]) # Boundary 1
        bc.append([0, 1, b+ww+r]) # Boundary 2
        bc.append([-1, 0, a-ww-r+ws[0]]) # Boundary 3
        bc.append([0, -1, b-ww-r+ws[1]]) # Boundary 4
        
        # Creating constraint of wall 1
        wc1=[]
        wc1.append([-1, 0, a-ww/2-r+ws[0]/4])
        wc1.append([0, 1, b+ww+ws[1]*3/4+r])
        wc1.append([1, 0, a+ww/2+r+ws[0]/4])
        
        # Creating constraint of wall 2
        wc2=[]
        wc2.append([-1, 0, a-ww/2-r+ws[0]/2])
        wc2.append([0, -1, b+ww-r+ws[1]/4])
        wc2.append([1, 0, a+ww/2+r+ws[0]/2])
        
        # Creating constraint of wall 3
        wc3=[]
        wc3.append([-1, 0, a-ww/2-r+ws[0]*3/4])
        wc3.append([0, 1, b+ww+r+ws[1]*3/4])
        wc3.append([1, 0, a+ww/2+r+ws[0]*3/4])
        
        return (bc,wc1,wc2,wc3)
        
    def check_collision(self,constraints,pose):
        print pose
        for i in range(len(constraints)):
            count = 0
            for j in range(len(constraints[i])): # bc needs count=4
                print "Now",constraints[i][j]
                ##################
                # Checking Boundary collision
                if i==0: # It is a boundary
                    if constraints[i][j][0]!=0: # Constraints with x-axis conditions
                        if constraints[i][j][0]*pose[0] <= constraints[i][j][2]: # Violation of constraints
                            print "xPose",pose[0]
                            print "Constraint",constraints[i][j][2]
                            print "Boundary Collision"
                            if constraints[i][j][0] >= 0: # Constraint type 1 (180 degrees)
                                print "First"
                                print constraints[i][j]
                                pose[2] = (pose[2] - 2*(pi-pose[2]))%(2*pi)
                            else:                         # Constraint type 2 (0 degrees)
                                print "Second"
                                print constraints[i][j]
                                pose[2] = (pose[2] + 2*(pose[2]))%(2*pi)
                    
                    if constraints[i][j][1]!=0: # Constraints with y-axis conditions
                        if constraints[i][j][1]*pose[1] <= constraints[i][j][2]: # Violation of constraints
                            print "yPose",pose[1]
                            print "Constraint",constraints[i][j][2]
                            print "Boundary Collision"
                            if constraints[i][j][1] >= 0: # Constraint type 1 (270 degrees)
                                print "Third"
                                print constraints[i][j]
                                pose[2] = (pose[2] - 2*(3*pi/2-pose[2]))%(2*pi)
                            else:
                                print "Fourth"
                                print constraints[i][j]
                                pose[2] = (pose[2] - 2*(pi/2-pose[2]))%(2*pi)
                
                ###################
                # Checking inner wall collision
                else:
                    if constraints[i][j][0]!=0: # Constraints with x-axis conditions
                        if constraints[i][j][0]*pose[0] <= constraints[i][j][2]: # Violation of constraints
                            count += 1
                            print "Constraint",constraints[i][j]
                            if fabs(fabs(pose[0]) - fabs(constraints[i][j][2])) < 1:
                                check = j
                
                    if constraints[i][j][1]!=0: # Constraints with y-axis conditions
                        if constraints[i][j][1]*pose[1] <= constraints[i][j][2]: # Violation of constraints
                            count += 1
                            print "Constraint",constraints[i][j]
                            if fabs(fabs(pose[1]) - fabs(constraints[i][j][2])) < 1:
                                check = -1*j
                                
                    if count == len(constraints[i]):
                        print "Inner wall collision detected"
                        print "Constraints",constraints[i]
                        if check > 0:
                            if constraints[i][check][0] <= 0: # Constraint type 1 (180 degrees)
                                pose[2] = (pose[2] - 2*(pi-pose[2]))%(2*pi)
                            else:                         # Constraint type 2 (0 degrees)
                                pose[2] = (pose[2] + 2*(pose[2]))%(2*pi)
                        else:
                            if constraints[i][-1*check][1] <= 0: # Constraint type 1 (270 degrees)
                                pose[2] = (pose[2] - 2*(3*pi/2-pose[2]))%(2*pi)
                            else:
                                pose[2] = (pose[2] - 2*(pi/2-pose[2]))%(2*pi)
        return pose   
                                
        
if __name__=="__main__":
    s = Sphero()
    pose = [100,150,0]
    control = [100,0.1]
    pose = s.motion(pose,control,0.1)
    print pose
    for i in range(10):
        pose = s.motion(pose,control,0.1)
        print pose

[109.99998476913288, 150.017453283659, 0.008726641829491543]
[119.99996676399059, 150.03642965111553, 0.017453283658983087]
[129.99994575259433, 150.0569291019294, 0.02617992548847463]
[139.9999215029654, 150.07895163562506, 0.034906567317966174]
[149.9998937831252, 150.10249725169163, 0.04363320914745772]
[159.99986236109513, 150.1275659495829, 0.052359850976949264]
[169.99982700489676, 150.15415772871734, 0.06108649280644081]
[179.99978748255168, 150.18227258847807, 0.06981313463593235]
[189.99974356208165, 150.21191052821288, 0.07853977646542389]
[199.99969501150846, 150.24307154723422, 0.08726641829491542]
[209.99964159885408, 150.27575564481924, 0.09599306012440696]


In [32]:
(-45)%360

315